In [1]:
from py2neo import Graph

In [2]:
g = Graph('http://localhost:7474', auth=('neo4j', '123456'), name='test')
g.delete_all()

In [3]:
cql_m = """create(:moive{name:'无间道', type:'犯罪', score:8.7}),
(:moive{name:'大内密探零零发', type:'搞笑', score:9.4}),
(:moive{name:'我和我的祖国', type:'主旋律', score:7.7})"""
g.run(cql_m)

(No data)

In [4]:
# 节点名称为a(相当于变量名,仅在该语句中有效)
cql_a = """match(a:moive)
return a"""

g.run(cql_a)

a
"{score: 8.7, name: '\u65e0\u95f4\u9053', type: '\u72af\u7f6a'}"
"{score: 9.4, name: '\u5927\u5185\u5bc6\u63a2\u96f6\u96f6\u53d1', type: '\u641e\u7b11'}"
"{score: 7.7, name: '\u6211\u548c\u6211\u7684\u7956\u56fd', type: '\u4e3b\u65cb\u5f8b'}"


In [5]:
g.run(cql_a).data()  # 列表,列表元素为字典,字典键名为CQL返回值变量名

[{'a': Node('moive', name='无间道', score=8.7, type='犯罪')},
 {'a': Node('moive', name='大内密探零零发', score=9.4, type='搞笑')},
 {'a': Node('moive', name='我和我的祖国', score=7.7, type='主旋律')}]

In [6]:
g.run("match(a:moive{name:'无间道'}) return a").data()

[{'a': Node('moive', name='无间道', score=8.7, type='犯罪')}]

In [7]:
# 查询多条记录(逗号分隔)
g.run("match(a:moive{name:'无间道'}), (b:moive{name:'我和我的祖国'}) return a, b").data()

[{'a': Node('moive', name='无间道', score=8.7, type='犯罪'),
  'b': Node('moive', name='我和我的祖国', score=7.7, type='主旋律')}]

In [8]:
g.run(cql_a).to_data_frame()

,a
0,"{'score': 8.7, 'name': '无间道', 'type': '犯罪'}"
1,"{'score': 9.4, 'name': '大内密探零零发', 'type': '搞笑'}"
2,"{'score': 7.7, 'name': '我和我的祖国', 'type': '主旋律'}"


In [9]:
g.run(cql_a).to_ndarray()

array([[Node('moive', name='无间道', score=8.7, type='犯罪')],
       [Node('moive', name='大内密探零零发', score=9.4, type='搞笑')],
       [Node('moive', name='我和我的祖国', score=7.7, type='主旋律')]],
      dtype=object)

In [10]:
cql_attr = """match(a:moive)
return a.type, a.name, a.score"""
g.run(cql_attr).data()

[{'a.type': '犯罪', 'a.name': '无间道', 'a.score': 8.7},
 {'a.type': '搞笑', 'a.name': '大内密探零零发', 'a.score': 9.4},
 {'a.type': '主旋律', 'a.name': '我和我的祖国', 'a.score': 7.7}]

In [11]:
# 返回所有节点
g.run("match (n) return n").data()

[{'n': Node('moive', name='无间道', score=8.7, type='犯罪')},
 {'n': Node('moive', name='大内密探零零发', score=9.4, type='搞笑')},
 {'n': Node('moive', name='我和我的祖国', score=7.7, type='主旋律')}]

In [12]:
g.run("create (:person{name:'xiaomai', age:4})-[:love{level:'must', time:5}]->(:moive{name:'西游记'})")
g.run("create (:person{name:'dmm', age:4})-[:dislike{level:'must', time:4}]->(:moive{name:'三国演义'})")

(No data)

In [13]:
# 指定关系类型为love
# 开始节点,结尾节点为任意
# 关系名称为r(相当于变量名,仅在该语句中有效)
g.run("""match ()-[r:love]->() return r""").data()

[{'r': love(Node('person', age=4, name='xiaomai'), Node('moive', name='西游记'), level='must', time=5)}]

In [14]:
# 指定关系time属性为4
# 开始节点,结尾节点为任意
g.run("""match ()-[r{time:4}]->() return r""").data()

[{'r': dislike(Node('person', age=4, name='dmm'), Node('moive', name='三国演义'), level='must', time=4)}]

In [15]:
# 关系为任意
# 开始节点,结尾节点为任意
g.run("""match ()-[r]->() return r""").data()

[{'r': love(Node('person', age=4, name='xiaomai'), Node('moive', name='西游记'), level='must', time=5)},
 {'r': dislike(Node('person', age=4, name='dmm'), Node('moive', name='三国演义'), level='must', time=4)}]

In [16]:
# 关系为任意
# 开始节点类型为person,结尾节点为任意
g.run("""match (:person)-[r]->() return r""").data()

[{'r': love(Node('person', age=4, name='xiaomai'), Node('moive', name='西游记'), level='must', time=5)},
 {'r': dislike(Node('person', age=4, name='dmm'), Node('moive', name='三国演义'), level='must', time=4)}]

In [17]:
# 关系为任意
# 开始节点类型name属性为xiaomai,结尾节点为任意
g.run("""match (:person{name:'xiaomai'})-[r]->() return r""").data()

[{'r': love(Node('person', age=4, name='xiaomai'), Node('moive', name='西游记'), level='must', time=5)}]

In [18]:
# 指定关系类型为love
# 开始节点类型name属性为xiaomai,结尾节点类型为moive
g.run("""match (:person{name:'xiaomai'})-[r:love]->(:moive) return r""").data()

[{'r': love(Node('person', age=4, name='xiaomai'), Node('moive', name='西游记'), level='must', time=5)}]

In [19]:
# 返回开始节点(cc),结尾节点(c),开始与结尾节点的关系(r)
g.run("match (cc)-[r]->(c) where r.time = 5 return cc, c, r").data()



[{'cc': Node('person', age=4, name='xiaomai'),
  'c': Node('moive', name='西游记'),
  'r': love(Node('person', age=4, name='xiaomai'), Node('moive', name='西游记'), level='must', time=5)}]